In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
dir0 = os.path.dirname(dir1)
if not dir1 in sys.path: sys.path.append(dir0)

In [21]:
from brio.utils.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from pickle import dump, load
import pandas as pd
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from functools import reduce
from itertools import compress

In [3]:
from brio.bias.FreqVsFreqBiasDetector import FreqVsFreqBiasDetector, BiasDetector
from brio.risk.HazardFromBiasDetectionCalculator import HazardFromBiasDetectionCalculator
from brio.risk.RiskCalculator import RiskCalculator

In [4]:
model_1 = pd.read_csv("./data/processed_model01.csv")
model_2 = pd.read_csv("./data/processed_model02.csv")
model_3 = pd.read_csv("./data/processed_model03.csv")
model_4 = pd.read_csv("./data/processed_model04.csv")
model_5 = pd.read_csv("./data/processed_model05.csv")

datasets = [model_1, model_2, model_3, model_4, model_5]
names = ["model_1", "model_2", "model_3", "model_4", "model_5"]

datasets_dict = {
    "model_1": model_1,
    "model_2": model_2,
    "model_3": model_3,
    "model_4": model_4,
    "model_5": model_5
}

In [14]:
sensitive_list = ['foreign_worker', 'personal_status_and_sex', 'age_group']
threshold=0.15
mode = 'group'
distance="TVD"
A1='high'
min_obs_per_group=30
aggregating_function=max
business_list_by_model = {"model_1": ['status', 'duration', 'credit_history'],
                          "model_2": ['status', 'duration', 'credit_history'],
                          "model_3": ['status', 'property', 'credit_history'],
                          "model_4": ['status', 'duration', 'credit_history'],
                          "model_5": ['credit_history', 'number_of_credits', 'employment_duration']                         }

## Risk Path con valori percentuali

In [25]:
path = {"step_1": [False, False, False],
        "step_2": [True, False, False],
        "step_3": [True, True, True],
        "step_4": [True, True, True]}

In [45]:
results = {}
for model in names:
    print(model)
    
    results[model] = {}

    conditioning_variables = business_list_by_model[model]
    print(conditioning_variables)
    
    for path_name, path_rules in path.items():

        path_variables = list(compress(conditioning_variables, path_rules))
        
        model_hazards_for_a_path = []
        for sensitive in sensitive_list:
            bias = bd.compare_root_variable_groups(
                    dataframe=datasets_dict["model_1"],
                    target_variable='predicted_label',
                    root_variable=sensitive,
                    threshold=threshold
                )  

            if len(path_variables) > 0:
                bias_conditioned = bd.compare_root_variable_conditioned_groups(
                        dataframe=datasets_dict["model_1"],
                        target_variable='predicted_label',
                        conditioning_variables=path_variables,
                        root_variable=sensitive,
                        threshold=threshold,
                        min_obs_per_group=min_obs_per_group
                    )
            else:
                bias_conditioned = None


            hazard = hc.compute_hazard_from_freqvsfreq_or_freqvsref(
                    overall_result=bias,
                    conditioned_results=bias_conditioned,
                    tot_observations=datasets_dict["model_1"].shape[0],
                    conditioning_variables=business_list_by_model["model_1"],
                    weight_logic=mode
                )
            
            model_hazards_for_a_path.append(hazard[0]/hazard[-1]) #just take the overall hazard and divides by max
            
        path_results_for_a_model = np.mean(model_hazards_for_a_path)
            
        results[model][path_name] = path_results_for_a_model   

model_1
['status', 'duration', 'credit_history']
model_2
['status', 'duration', 'credit_history']
model_3
['status', 'property', 'credit_history']
model_4
['status', 'duration', 'credit_history']
model_5
['credit_history', 'number_of_credits', 'employment_duration']


In [51]:
pd.DataFrame(results)

,model_1,model_2,model_3,model_4,model_5
step_1,1.000000,1.000000,1.000000,1.000000,1.000000
step_2,0.700924,0.700924,0.700924,0.700924,0.823698
step_3,0.758811,0.758811,0.746344,0.758811,0.815824
step_4,0.758811,0.758811,0.746344,0.758811,0.815824


## Risk Path con valori assoluti

In [52]:
path = {"step_1": [False, False, False],
        "step_2": [True, False, False],
        "step_3": [True, True, True],
        "step_4": [True, True, True]}

In [53]:
results = {}
for model in names:
    print(model)
    
    results[model] = {}

    conditioning_variables = business_list_by_model[model]
    print(conditioning_variables)
    
    for path_name, path_rules in path.items():

        path_variables = list(compress(conditioning_variables, path_rules))
        
        model_hazards_for_a_path = []
        for sensitive in sensitive_list:
            bias = bd.compare_root_variable_groups(
                    dataframe=datasets_dict["model_1"],
                    target_variable='predicted_label',
                    root_variable=sensitive,
                    threshold=threshold
                )  

            if len(path_variables) > 0:
                bias_conditioned = bd.compare_root_variable_conditioned_groups(
                        dataframe=datasets_dict["model_1"],
                        target_variable='predicted_label',
                        conditioning_variables=path_variables,
                        root_variable=sensitive,
                        threshold=threshold,
                        min_obs_per_group=min_obs_per_group
                    )
            else:
                bias_conditioned = None


            hazard = hc.compute_hazard_from_freqvsfreq_or_freqvsref(
                    overall_result=bias,
                    conditioned_results=bias_conditioned,
                    tot_observations=datasets_dict["model_1"].shape[0],
                    conditioning_variables=business_list_by_model["model_1"],
                    weight_logic=mode
                )
            
            model_hazards_for_a_path.append(hazard[0]) #just take the overall hazard and divides by max
            
        path_results_for_a_model = np.mean(model_hazards_for_a_path)
            
        results[model][path_name] = path_results_for_a_model   

model_1
['status', 'duration', 'credit_history']
model_2
['status', 'duration', 'credit_history']
model_3
['status', 'property', 'credit_history']
model_4
['status', 'duration', 'credit_history']
model_5
['credit_history', 'number_of_credits', 'employment_duration']


In [54]:
pd.DataFrame(results)

,model_1,model_2,model_3,model_4,model_5
step_1,0.150128,0.150128,0.150128,0.150128,0.150128
step_2,0.079050,0.079050,0.079050,0.079050,0.108424
step_3,0.062602,0.062602,0.060277,0.062602,0.064025
step_4,0.062602,0.062602,0.060277,0.062602,0.064025
